In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr


In [2]:
polling_df = pd.read_csv('../election_resources/cleaned_2019_data.csv')
polling_df.head()

,question_id,end_date,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,114577,2019-12-10,YouGov,1195,rv,Michael F. Bennet,14.0,29.0,4.0,10.0,12.0,17.0,0
1,114585,2019-12-10,YouGov,1198,rv,Michael Bloomberg,24.0,58.0,7.0,17.0,20.0,38.0,0
2,114589,2019-12-10,YouGov,1198,rv,Cory A. Booker,35.0,45.0,11.0,24.0,12.0,33.0,0
3,114593,2019-12-10,YouGov,1198,rv,Steve Bullock,14.0,27.0,3.0,11.0,11.0,16.0,0
4,114597,2019-12-10,YouGov,1202,rv,Pete Buttigieg,35.0,44.0,14.0,21.0,13.0,31.0,0


In [3]:
polling_df.drop(columns=['question_id', 'end_date'], inplace=True)
polling_df.head()

,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,YouGov,1195,rv,Michael F. Bennet,14.0,29.0,4.0,10.0,12.0,17.0,0
1,YouGov,1198,rv,Michael Bloomberg,24.0,58.0,7.0,17.0,20.0,38.0,0
2,YouGov,1198,rv,Cory A. Booker,35.0,45.0,11.0,24.0,12.0,33.0,0
3,YouGov,1198,rv,Steve Bullock,14.0,27.0,3.0,11.0,11.0,16.0,0
4,YouGov,1202,rv,Pete Buttigieg,35.0,44.0,14.0,21.0,13.0,31.0,0


In [4]:
polling_df.nunique()

pollster                 13
sample_size             288
population                1
politician               34
favorable                68
unfavorable              65
very_favorable           49
somewhat_favorable       43
somewhat_unfavorable     33
very_unfavorable         60
results                   2
dtype: int64

In [6]:
polling_df['pollster'].value_counts()

YouGov                          949
HarrisX                         271
Morning Consult                 195
Harris Poll                      68
Quinnipiac University            48
CNN/SSRS                         46
Monmouth University              19
Fox News                         17
ABC News/Washington Post          6
Marist College                    5
Suffolk University                4
Public Policy Polling             2
NBC News/Wall Street Journal      1
Name: pollster, dtype: int64

In [7]:
pollster_to_replace = list(polling_df['pollster'].value_counts()[polling_df['pollster'].value_counts() < 10].index)
for pollster in pollster_to_replace:
    polling_df['pollster'] = polling_df['pollster'].replace(pollster, 'pollster<10')
polling_df['pollster'].value_counts()

YouGov                   949
HarrisX                  271
Morning Consult          195
Harris Poll               68
Quinnipiac University     48
CNN/SSRS                  46
Monmouth University       19
pollster<10               18
Fox News                  17
Name: pollster, dtype: int64

In [18]:
polling_df['politician'].value_counts()

Donald Trump             135
Elizabeth Warren         101
Bernard Sanders           99
Joseph R. Biden Jr.       95
Kamala D. Harris          91
Pete Buttigieg            88
Cory A. Booker            77
Amy Klobuchar             75
Julián Castro             72
Beto O'Rourke             71
Tulsi Gabbard             66
Andrew Yang               63
John K. Delaney           62
Kirsten E. Gillibrand     50
Marianne Williamson       46
Tim Ryan                  41
Michael F. Bennet         41
Wayne Messam              40
Steve Bullock             40
Jay Robert Inslee         40
John Hickenlooper         39
Tom Steyer                32
Bill de Blasio            30
Joe Sestak                29
Seth Moulton              26
Eric Swalwell             22
Mike Gravel               21
Michael Bloomberg         12
Sherrod Brown              8
Stacey Yvonne Abrams       7
Deval Patrick              5
Terry R. McAuliffe         3
Eric H. Holder             2
Eric Garcetti              2
Name: politici

In [19]:
politician_to_replace = list(polling_df['politician'].value_counts()[polling_df['politician'].value_counts() < 10].index)
for i in politician_to_replace:
    polling_df['politician'] = polling_df['politician'].replace(i, 'politician<10')
polling_df['politician'].value_counts()

Donald Trump             135
Elizabeth Warren         101
Bernard Sanders           99
Joseph R. Biden Jr.       95
Kamala D. Harris          91
Pete Buttigieg            88
Cory A. Booker            77
Amy Klobuchar             75
Julián Castro             72
Beto O'Rourke             71
Tulsi Gabbard             66
Andrew Yang               63
John K. Delaney           62
Kirsten E. Gillibrand     50
Marianne Williamson       46
Tim Ryan                  41
Michael F. Bennet         41
Wayne Messam              40
Steve Bullock             40
Jay Robert Inslee         40
John Hickenlooper         39
Tom Steyer                32
Bill de Blasio            30
Joe Sestak                29
politician<10             27
Seth Moulton              26
Eric Swalwell             22
Mike Gravel               21
Michael Bloomberg         12
Name: politician, dtype: int64

In [4]:
polling_df = pd.get_dummies(polling_df, dtype=float)
polling_df.head()

,sample_size,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results,pollster_ABC News/Washington Post,pollster_CNN/SSRS,...,politician_Pete Buttigieg,politician_Seth Moulton,politician_Sherrod Brown,politician_Stacey Yvonne Abrams,politician_Steve Bullock,politician_Terry R. McAuliffe,politician_Tim Ryan,politician_Tom Steyer,politician_Tulsi Gabbard,politician_Wayne Messam
0,1195,14.0,29.0,4.0,10.0,12.0,17.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1198,24.0,58.0,7.0,17.0,20.0,38.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1198,35.0,45.0,11.0,24.0,12.0,33.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1198,14.0,27.0,3.0,11.0,11.0,16.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1202,35.0,44.0,14.0,21.0,13.0,31.0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
polling_df.shape

(1631, 56)

In [5]:
polling_df.fillna(0, inplace=True)

In [6]:
y = polling_df['results'].values
X = polling_df.drop(['results'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

In [7]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu', 'selu'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                        min_value=1,
                                        max_value=100,
                                        step=2), activation=activation, input_dim=len(X_train_scaled[0])))
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                        min_value=1,
                                        max_value=100,
                                        step=2), activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    loss = hp.Choice('loss', ['binary_crossentropy', 'mse'])
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam'])
    nn_model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    
    return nn_model

In [20]:
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=10, directory='my_dir2', project_name='intro_to_kt')

In [21]:
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 300 Complete [00h 00m 10s]
val_accuracy: 0.9975489974021912

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 32m 52s
INFO:tensorflow:Oracle triggered exit


In [22]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 75,
 'num_layers': 8,
 'units_0': 15,
 'loss': 'mse',
 'optimizer': 'adam',
 'units_1': 67,
 'units_2': 37,
 'units_3': 61,
 'units_4': 5,
 'units_5': 29,
 'units_6': 83,
 'units_7': 13,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [23]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 4.2026e-04 - accuracy: 1.0000 - 258ms/epoch - 20ms/step
Loss: 0.00042026155279017985, Accuracy: 1.0


In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  75
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 67
hidden_nodes_layer4 = 37
hidden_nodes_layer5 = 61
hidden_nodes_layer6 = 5
hidden_nodes_layer7 = 29
hidden_nodes_layer8 = 83


output_dim = 1



nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer8, activation="tanh"))





# Output layer
nn_model.add(tf.keras.layers.Dense(output_dim, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 75)                4200      
                                                                 
 dense_10 (Dense)            (None, 15)                1140      
                                                                 
 dense_11 (Dense)            (None, 67)                1072      
                                                                 
 dense_12 (Dense)            (None, 37)                2516      
                                                                 
 dense_13 (Dense)            (None, 61)                2318      
                                                                 
 dense_14 (Dense)            (None, 5)                 310       
                                                                 
 dense_15 (Dense)            (None, 29)               

In [21]:
nn_model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])

In [22]:

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, initial_epoch= 0)

Epoch 1/100
39/39 [==============================] - 1s 6ms/step - loss: 0.0782 - accuracy: 0.9125
Epoch 2/100
39/39 [==============================] - 0s 5ms/step - loss: 0.0028 - accuracy: 0.9967
Epoch 3/100
39/39 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 0.9975
Epoch 4/100
39/39 [==============================] - 0s 6ms/step - loss: 9.3192e-04 - accuracy: 0.9992
Epoch 5/100
39/39 [==============================] - 0s 5ms/step - loss: 8.8227e-04 - accuracy: 0.9992
Epoch 6/100
39/39 [==============================] - 0s 6ms/step - loss: 8.6100e-04 - accuracy: 0.9992
Epoch 7/100
39/39 [==============================] - 0s 5ms/step - loss: 8.4737e-04 - accuracy: 0.9992
Epoch 8/100
39/39 [==============================] - 0s 5ms/step - loss: 8.3559e-04 - accuracy: 0.9992
Epoch 9/100
39/39 [==============================] - 0s 5ms/step - loss: 8.1833e-04 - accuracy: 0.9992
Epoch 10/100
39/39 [==============================] - 0s 5ms/step - loss: 6.9889e-04 

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 2.2734e-05 - accuracy: 1.0000 - 167ms/epoch - 13ms/step
Loss: 2.2734355297870934e-05, Accuracy: 1.0


In [24]:
nn_model.predict(X_train_scaled)

39/39 [==============================] - 0s 2ms/step


array([[9.9918586e-01],
       [4.3091210e-04],
       [4.4543494e-04],
       ...,
       [4.6053654e-04],
       [3.7544593e-04],
       [4.5514753e-04]], dtype=float32)

In [25]:
nn_model.predict(X_test_scaled)

13/13 [==============================] - 0s 2ms/step


array([[5.8571546e-04],
       [4.0246508e-04],
       [9.5150068e-02],
       [5.2911014e-04],
       [3.7475827e-04],
       [5.6484027e-04],
       [4.3369987e-04],
       [4.5888379e-04],
       [4.6640637e-04],
       [5.8739685e-04],
       [4.1079253e-04],
       [3.6048694e-04],
       [9.9918681e-01],
       [3.5586534e-04],
       [3.9764674e-04],
       [3.6352649e-04],
       [4.5809775e-04],
       [3.6299106e-04],
       [9.9917537e-01],
       [9.9914300e-01],
       [3.9343358e-04],
       [3.9423609e-04],
       [4.0954188e-04],
       [3.6260908e-04],
       [4.8011233e-04],
       [4.1510104e-04],
       [4.8108309e-04],
       [9.9821162e-01],
       [3.6545220e-04],
       [6.0357421e-04],
       [3.8279538e-04],
       [3.4829564e-04],
       [4.2699822e-04],
       [5.0150044e-04],
       [3.6522213e-04],
       [4.8764117e-04],
       [3.8374498e-04],
       [9.9918574e-01],
       [4.1209176e-04],
       [3.9275683e-04],
       [4.5071301e-04],
       [3.592498

In [13]:
new_df = pd.read_csv('../election_resources/cleaned_2023_data.csv')
new_df.head()

,question_id,end_date,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,178640,2023-08-08,YouGov,1500.0,a,Donald Trump,41.0,56.0,21.0,20.0,12.0,44.0,0
1,178641,2023-08-08,YouGov,1178.0,rv,Donald Trump,41.0,58.0,23.0,18.0,11.0,47.0,0
2,178646,2023-08-08,YouGov,1500.0,a,Joe Biden,44.0,53.0,20.0,24.0,13.0,40.0,0
3,178647,2023-08-08,YouGov,1178.0,rv,Joe Biden,45.0,53.0,23.0,22.0,11.0,42.0,0
4,178652,2023-08-08,YouGov,1500.0,a,Mike Pence,27.0,58.0,6.0,21.0,27.0,31.0,0


In [14]:
new_df.drop(columns = ['end_date', 'question_id', 'results'])

,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable
0,YouGov,1500.0,a,Donald Trump,41.0,56.0,21.0,20.0,12.0,44.0
1,YouGov,1178.0,rv,Donald Trump,41.0,58.0,23.0,18.0,11.0,47.0
2,YouGov,1500.0,a,Joe Biden,44.0,53.0,20.0,24.0,13.0,40.0
3,YouGov,1178.0,rv,Joe Biden,45.0,53.0,23.0,22.0,11.0,42.0
4,YouGov,1500.0,a,Mike Pence,27.0,58.0,6.0,21.0,27.0,31.0
...,...,...,...,...,...,...,...,...,...,...
2374,Morning Consult,2200.0,a,Donald Trump,42.0,54.0,26.0,16.0,9.0,45.0
2375,YouGov,1500.0,rv,Joe Biden,49.0,50.0,33.0,16.0,13.0,37.0
2376,YouGov,1500.0,rv,Donald Trump,45.0,54.0,33.0,12.0,7.0,47.0
2377,YouGov,1500.0,rv,Mike Pence,44.0,51.0,31.0,13.0,12.0,39.0


In [30]:
new_df = pd.get_dummies(polling_df, dtype=float)
new_df.describe()

,sample_size,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results,pollster_ABC News/Washington Post,pollster_CNN/SSRS,...,politician_Pete Buttigieg,politician_Seth Moulton,politician_Sherrod Brown,politician_Stacey Yvonne Abrams,politician_Steve Bullock,politician_Terry R. McAuliffe,politician_Tim Ryan,politician_Tom Steyer,politician_Tulsi Gabbard,politician_Wayne Messam
count,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,...,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000,1631.000000
mean,1263.738197,27.369651,32.833476,9.097977,14.003127,9.370386,18.510300,0.141018,0.003679,0.028204,...,0.053955,0.015941,0.004905,0.004292,0.024525,0.001839,0.025138,0.019620,0.040466,0.024525
std,347.533032,12.322381,12.386363,8.154154,7.986476,5.068035,12.287341,0.348147,0.060559,0.165605,...,0.225997,0.125286,0.069885,0.065391,0.154719,0.042861,0.156592,0.138733,0.197110,0.154719
min,442.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1072.500000,17.000000,22.000000,3.000000,9.000000,7.000000,10.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1181.000000,27.000000,33.000000,6.000000,14.000000,10.000000,17.800000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1268.000000,38.000000,42.000000,13.000000,21.000000,13.000000,27.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2531.000000,56.000000,58.000000,35.000000,33.000000,48.000000,52.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
polling_df.shape

(1631, 56)

In [16]:
polling_df.fillna(0, inplace=True)

In [17]:
scaler = StandardScaler()
X_scaler = scaler.fit(new_df)
X_scaled = X_scaler.transform(new_df)


In [26]:
nn_model.predict(X_scaled)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/dense_9/MatMul' defined at (most recent call last):
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\micha\anaco